In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('2025-10-24/direction_walk_forward___detailed.csv')
df_wf = df[df['fold_type'] == 'walk_forward'].copy()

# 모델별 Fold 간 변동성 분석
consistency_metrics = []
for model in df_wf['Model'].unique():
    model_data = df_wf[df_wf['Model'] == model]
    consistency_metrics.append({
        'Model': model,
        'Avg_Train': model_data['Train_Accuracy'].mean(),
        'Avg_Val': model_data['Val_Accuracy'].mean(),
        'Avg_Test': model_data['Test_Accuracy'].mean(),
        'Std_Test': model_data['Test_Accuracy'].std(),
        'Min_Test': model_data['Test_Accuracy'].min(),
        'Max_Test': model_data['Test_Accuracy'].max(),
        'Test_Range': model_data['Test_Accuracy'].max() - model_data['Test_Accuracy'].min(),
    })

consistency_df = pd.DataFrame(consistency_metrics)

# 과적합 분석
overfitting_analysis = []
for model in df_wf['Model'].unique():
    model_data = df_wf[df_wf['Model'] == model]
    for idx, row in model_data.iterrows():
        train_test_gap = row['Train_Accuracy'] - row['Test_Accuracy']
        val_test_gap = abs(row['Val_Accuracy'] - row['Test_Accuracy'])
        overfitting_analysis.append({
            'Model': model,
            'Fold': row['Fold'],
            'Train': row['Train_Accuracy'],
            'Val': row['Val_Accuracy'],
            'Test': row['Test_Accuracy'],
            'Train_Test_Gap': train_test_gap,
            'Val_Test_Gap': val_test_gap,
            'Overfitted_0.10': train_test_gap > 0.10,
            'Overfitted_0.15': train_test_gap > 0.15
        })

overfit_df = pd.DataFrame(overfitting_analysis)

# 모델별 과적합 요약
overfit_summary = overfit_df.groupby('Model').agg({
    'Overfitted_0.10': 'sum',
    'Overfitted_0.15': 'sum',
    'Train_Test_Gap': ['mean', 'max', 'std'],
    'Val_Test_Gap': 'mean'
}).round(4)

overfit_summary.columns = ['Overfit_0.10_Count', 'Overfit_0.15_Count', 
                            'Avg_Gap', 'Max_Gap', 'Std_Gap', 'Avg_ValTest_Gap']

# 종합 평가
final_evaluation = consistency_df.merge(overfit_summary, left_on='Model', right_index=True)

# 안정성 점수 (낮을수록 좋음)
final_evaluation['Stability_Score'] = (
    final_evaluation['Std_Test'] * 2 +
    final_evaluation['Test_Range'] +
    final_evaluation['Avg_Gap'] +
    final_evaluation['Overfit_0.10_Count'] * 0.05
)

final_evaluation = final_evaluation.sort_values('Stability_Score')

# 추천 모델 (암호화폐 트레이딩 기준)
recommended = final_evaluation[
    (final_evaluation['Std_Test'] < 0.08) &
    (final_evaluation['Overfit_0.10_Count'] <= 2) &
    (final_evaluation['Avg_Test'] >= 0.58) &
    (final_evaluation['Max_Gap'] < 0.20)
].copy()

# CSV 저장
final_evaluation.to_csv('model_evaluation_complete.csv', index=False)
overfit_df.to_csv('fold_by_fold_analysis.csv', index=False)
recommended.to_csv('recommended_models.csv', index=False)

print("\n=== 전체 모델 평가 (안정성 점수 기준 정렬) ===")
print(final_evaluation[['Model', 'Avg_Test', 'Std_Test', 'Overfit_0.10_Count', 
                         'Max_Gap', 'Stability_Score']].to_string(index=False))

print("\n\n=== 추천 모델 (실전 트레이딩 기준) ===")
if len(recommended) > 0:
    print(recommended[['Model', 'Avg_Test', 'Std_Test', 'Min_Test', 'Max_Test', 
                       'Overfit_0.10_Count', 'Stability_Score']].to_string(index=False))
else:
    print("기준을 만족하는 모델 없음")
    
print("\n\n=== Top 5 모델 Fold별 상세 ===")
for model in final_evaluation.head(5)['Model']:
    model_detail = overfit_df[overfit_df['Model'] == model][
        ['Fold', 'Train', 'Val', 'Test', 'Train_Test_Gap']
    ].sort_values('Fold')
    print(f"\n{model}:")
    print(model_detail.to_string(index=False))

print("\n\nFiles saved:")
print("- model_evaluation_complete.csv")
print("- fold_by_fold_analysis.csv")
print("- recommended_models.csv")



=== 전체 모델 평가 (안정성 점수 기준 정렬) ===
             Model  Avg_Test  Std_Test  Overfit_0.10_Count  Max_Gap  Stability_Score
LogisticRegression  0.625926  0.028689                   0   0.0818         0.164344
        NaiveBayes  0.507407  0.045361                   2   0.1282         0.363944
          DTW_LSTM  0.569444  0.048781                   2   0.1218         0.369330
              LSTM  0.555556  0.044305                   2   0.1535         0.386077
            TabNet  0.477778  0.061262                   1   0.1537         0.409492
          CatBoost  0.646296  0.032521                   3   0.2934         0.437531
   Hybrid_LSTM_GRU  0.541667  0.056519                   2   0.1570         0.437839
            BiLSTM  0.552778  0.037143                   4   0.1684         0.476787
          LightGBM  0.646296  0.044119                   4   0.1855         0.521349
               SVM  0.596296  0.026912                   5   0.2353         0.524003
          AdaBoost  0.598148  0.

In [4]:
import pandas as pd
import os

# --- 👇 모든 행과 열을 출력하도록 설정 변경 👇 ---
pd.set_option('display.max_rows', None)    # 행 제한 해제
pd.set_option('display.max_columns', None) # 열 제한 해제 (필요하다면 추가)
# ----------------------------------------------------

# 파일 목록
file_names = [
    "direction_return_walk_forward.csv"
#     "direction_return_tvt.csv",
#     "direction_tvt.csv",
#     "direction_tvt.csv",
#     "return_walk_forward.csv",
#     "return_tvt.csv"
]

# DataFrame을 저장할 딕셔너리
dataframes = {}

# 각 파일을 읽어서 DataFrame으로 저장하고 모든 행 출력
for file_name in file_names:
    try:
        # DataFrame 이름 생성 (확장자 제거)
        df_name = file_name.replace(".csv", "")
        
        # CSV 파일 읽기
        df = pd.read_csv(file_name)
        
        # 딕셔너리에 저장
        dataframes[df_name] = df
        
        df=df.sort_values(by="Test_Accuracy", ascending=False)
        
        print(f"--- {file_name} 파일 확인 (모든 행 출력) ---")
        # 설정 변경 후 print(df)를 사용하면 모든 행이 출력됩니다.
        print(df) 
        print("-" * (len(file_name) + 30))
        
    except FileNotFoundError:
        print(f"오류: {file_name} 파일을 찾을 수 없습니다. 경로를 확인해주세요.")
    except Exception as e:
        print(f"오류: {file_name} 파일을 읽는 중 오류가 발생했습니다: {e}")

--- direction_return_walk_forward.csv 파일 확인 (모든 행 출력) ---
                             Model  Train_Accuracy  Val_Accuracy  \
42          Ensemble_MTL_task1_cls        0.818182      0.818182   
21           CNN_GRU_MTL_task1_cls        0.772727      0.772727   
9   SoftSharing_LSTM_MTL_task1_cls        0.772727      0.772727   
18          CNN_LSTM_MTL_task1_cls        0.727273      0.727273   
33               TCN_MTL_task1_cls        0.727273      0.727273   
36      Hierarchical_MTL_task1_cls        0.727273      0.727273   
75     MetaRL_Crypto_MTL_task1_cls        0.727273      0.727273   
66              SNAS_MTL_task1_cls        0.727273      0.727273   
3             BiLSTM_MTL_task1_cls        0.681818      0.681818   
0   HardSharing_LSTM_MTL_task1_cls        0.681818      0.681818   
6                GRU_MTL_task1_cls        0.681818      0.681818   
63       EtherVoyant_MTL_task1_cls        0.681818      0.681818   
39      WeightedLoss_MTL_task1_cls        0.681818      0.